<a href="https://colab.research.google.com/github/jrleonett/Error-ELA-and-Imagen-MetadataImagen/blob/main/Error_ELA_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análisis de metadatos y Error de ELA en imágenes digitales**

Este programa es una herramienta de computación forense diseñado para analizar imágenes en busca de evidencia de manipulación o edición. Utiliza la técnica de **Error Level Analysis** (**ELA**) y, este programa fué desarrollado por José R. Leonett para los peritos forenses digitales, analistas forenses o cualquier persona interesada en verificar la autenticidad de imágenes digitales.

Este programa fué desarrollado por **José R. Leonett** para los peritos forenses digitales de guatemala www.forensedigital.gt, analistas forenses o cualquier persona interesada en verificar la autenticidad de imágenes digitales.

In [6]:
#@title Configuración del entorno Gradio e Instalación de las bibliotecas necesarias

# Instalar dependencias necesarias
!pip install gradio pillow opencv-python pyngrok

import cv2
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import hashlib
import gradio as gr
import matplotlib.pyplot as plt
import tempfile
import os
from pyngrok import ngrok
from datetime import datetime

# Configurar el token de autenticación de ngrok
ngrok.set_auth_token("2roOWxB4cG9hk3jcnZ0EdjFvB8j_6z8MGd6zDj6pv9efMgKVQ")

def obtener_metadatos(imagen):
    """
    Obtiene los metadatos de la imagen.
    """
    metadatos = {}
    try:
        exif_data = imagen._getexif()
        if exif_data:
            for tag_id, value in exif_data.items():
                tag = TAGS.get(tag_id, tag_id)
                metadatos[tag] = value
    except Exception as e:
        print(f"Error al obtener metadatos: {e}")
    return metadatos

def obtener_coordenadas(exif_data):
    """
    Obtiene las coordenadas GPS de los metadatos EXIF.
    """
    if not exif_data:
        return None

    gps_info = exif_data.get("GPSInfo", {})
    if not gps_info:
        return None

    # Convertir las coordenadas GPS a grados decimales
    def gps_to_degrees(value):
        d, m, s = value
        return d + (m / 60.0) + (s / 3600.0)

    try:
        latitud = gps_info.get(2)  # Latitud
        longitud = gps_info.get(4)  # Longitud
        latitud_ref = gps_info.get(1)  # Norte/Sur
        longitud_ref = gps_info.get(3)  # Este/Oeste

        if latitud and longitud and latitud_ref and longitud_ref:
            latitud = gps_to_degrees(latitud)
            longitud = gps_to_degrees(longitud)
            if latitud_ref == "S":  # Si es Sur, convertir a negativo
                latitud = -latitud
            if longitud_ref == "W":  # Si es Oeste, convertir a negativo
                longitud = -longitud
            return latitud, longitud
    except Exception as e:
        print(f"Error al procesar coordenadas GPS: {e}")
        return None

    return None

def calcular_hash(imagen):
    """
    Calcula el hash MD5 de la imagen para verificar su integridad.
    """
    imagen_bytes = imagen.tobytes()
    return hashlib.md5(imagen_bytes).hexdigest()

def analizar_manipulacion(imagen, metadatos):
    """
    Analiza si la imagen ha sido manipulada.
    """
    manipulada = False
    razones = []

    # Verificar si la imagen tiene marcas de agua
    if imagen.mode == "P":
        razones.append("La imagen tiene marcas de agua o es una imagen indexada.")
        manipulada = True

    # Verificar si los metadatos han sido alterados
    if not metadatos:
        razones.append("La imagen no tiene metadatos EXIF.")
        manipulada = True
    else:
        if "Software" in metadatos:
            razones.append(f"La imagen fue editada con: {metadatos['Software']}")
            manipulada = True

    # Verificar si el hash de la imagen coincide con un hash conocido (simulación)
    hash_conocido = "1a79a4d60de6718e8e5b326e338ae533"  # Hash de ejemplo
    hash_actual = calcular_hash(imagen)
    if hash_actual != hash_conocido:
        razones.append("El hash de la imagen no coincide con el hash conocido.")
        manipulada = True

    return manipulada, razones

def realizar_ela(imagen, quality=95, scale=100):
    """
    Realiza el Error Level Analysis (ELA) en la imagen utilizando OpenCV.
    """
    # Convertir la imagen a formato compatible con OpenCV
    imagen_cv = cv2.cvtColor(np.array(imagen), cv2.COLOR_RGB2BGR)

    # Guardar la imagen comprimida
    temp_path = "temp_image.jpg"
    cv2.imwrite(temp_path, imagen_cv, [cv2.IMWRITE_JPEG_QUALITY, quality])

    # Leer la imagen comprimida
    imagen_comprimida = cv2.imread(temp_path)

    # Calcular la diferencia absoluta entre la imagen original y la comprimida
    diferencia = cv2.absdiff(imagen_cv, imagen_comprimida)

    # Escalar la diferencia para resaltar las áreas modificadas
    ela_imagen = scale * diferencia

    # Convertir a escala de grises para mejorar la visualización
    ela_imagen = cv2.cvtColor(ela_imagen, cv2.COLOR_BGR2GRAY)

    # Eliminar el archivo temporal
    os.remove(temp_path)

    return ela_imagen

def procesar_imagen(archivo_imagen):
    # Cargar la imagen
    try:
        imagen = Image.open(archivo_imagen)
    except Exception as e:
        return f"Error al cargar la imagen: {e}", None

    # Obtener información básica de la imagen
    nombre_archivo = os.path.basename(archivo_imagen)
    info_basica = f"""
    Información básica de la imagen:
    Nombre del archivo: {nombre_archivo}
    Formato: {imagen.format}
    Tamaño: {imagen.size} píxeles
    Modo: {imagen.mode}
    """

    # Obtener tamaño del archivo, fecha de creación y modificación
    stats = os.stat(archivo_imagen)
    tamaño_archivo = stats.st_size / 1024  # Tamaño en KB
    fecha_creacion = datetime.fromtimestamp(stats.st_ctime).strftime('%Y-%m-%d %H:%M:%S')
    fecha_modificacion = datetime.fromtimestamp(stats.st_mtime).strftime('%Y-%m-%d %H:%M:%S')

    info_basica += f"""
    Tamaño del archivo: {tamaño_archivo:.2f} KB
    Fecha de creación: {fecha_creacion}
    Fecha de modificación: {fecha_modificacion}
    """

    # Obtener los primeros 10 bytes del archivo en formato hexadecimal
    with open(archivo_imagen, "rb") as f:
        primeros_10_bytes = f.read(10)
        header_hex = " ".join(f"{byte:02x}" for byte in primeros_10_bytes)

    info_basica += f"""
    Primeros 10 bytes del archivo (hex): {header_hex}
    """

    # Obtener metadatos
    metadatos = obtener_metadatos(imagen)
    info_metadatos = "\n" + "-" * 50 + "\nANÁLISIS FORENSE DE LOS METADATOS DE LA IMAGEN:\n"
    if metadatos:
        for tag, value in metadatos.items():
            if tag == "DateTime":
                info_metadatos += f"Fecha y hora de la captura: {value}\n"
            elif tag == "Make":
                info_metadatos += f"Fabricante de la cámara: {value}\n"
            elif tag == "Model":
                info_metadatos += f"Modelo de la cámara: {value}\n"
            elif tag == "Software":
                info_metadatos += f"Software utilizado para editar la imagen: {value}\n"
            elif tag == "ExifImageWidth":
                info_metadatos += f"Ancho de la imagen: {value} píxeles\n"
            elif tag == "ExifImageHeight":
                info_metadatos += f"Alto de la imagen: {value} píxeles\n"
            elif tag == "GPSInfo":
                info_metadatos += "Información de ubicación GPS:\n"
                coordenadas = obtener_coordenadas(metadatos)
                if coordenadas:
                    latitud, longitud = coordenadas
                    enlace_google_maps = f"https://www.google.com/maps?q={latitud},{longitud}"
                    info_metadatos += f"- Coordenadas GPS: {latitud}, {longitud}\n"
                    info_metadatos += f"- Enlace a Google Maps: {enlace_google_maps}\n"
                else:
                    info_metadatos += "- No se encontraron coordenadas GPS.\n"
            else:
                info_metadatos += f"{tag}: {value}\n"
    else:
        info_metadatos += "No se encontraron metadatos.\n"

    # Analizar si la imagen ha sido manipulada
    manipulada, razones = analizar_manipulacion(imagen, metadatos)
    info_manipulacion = "\nAnálisis de manipulación:\n"
    if manipulada:
        info_manipulacion += "La imagen ha sido manipulada.\n"
        info_manipulacion += "Razones:\n"
        for razon in razones:
            info_manipulacion += f"- {razon}\n"
    else:
        info_manipulacion += "La imagen NO ha sido manipulada.\n"

    # Realizar Error Level Analysis (ELA)
    ela_imagen = realizar_ela(imagen)

    # Guardar la imagen ELA en un archivo temporal
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmp_file:
        cv2.imwrite(tmp_file.name, ela_imagen)
        ela_image_path = tmp_file.name

    return ela_image_path, info_basica + info_metadatos + info_manipulacion

# Interfaz de GRADIO
iface = gr.Interface(
    fn=procesar_imagen,
    inputs=gr.Image(type="filepath", label="Sube una imagen"),
    outputs=[gr.Image(label="Error Level Analysis (ELA)"), gr.Textbox(label="Resultado del análisis")],
    title="Análisis de metadatos y Error de ELA en imágenes digitales con Gradio",
    description="""<div style="text-align: center;">
    <p>Este programa es una herramienta de computación forense diseñado para analizar imágenes en busca de evidencia de manipulación o edición. Utiliza la técnica de Error Level Analysis (ELA) y fue desarrollado por José R. Leonett para los peritos forenses digitales de Guatemala <a href="http://www.forensedigital.gt">www.forensedigital.gt</a>, analistas forenses o cualquier persona interesada en verificar la autenticidad de imágenes digitales.</p>
    </div>"""
)

# Lanzar la interfaz de Gradio
print("Iniciando la aplicación Gradio...")
iface.launch(share=True)  # Forzar share=True para Colab

# Conectar con ngrok
print("\nGenerando enlace ngrok...")
ngrok_tunnel = ngrok.connect(addr="7860", proto="http")
ngrok_url = ngrok_tunnel.public_url
print(f"Enlace ngrok: {ngrok_url}")

Iniciando la aplicación Gradio...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06373bfdf96c2c5d77.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



Generando enlace ngrok...
Enlace ngrok: https://c544-34-125-243-86.ngrok-free.app
